# Imports

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

from coinmetrics.api_client import CoinMetricsClient
import logging

import yfinance as yf

from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm

# Get Data

## Get Bitcoin

In [5]:
# Configure logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Initialize Coin Metrics API client
client = CoinMetricsClient()

# Define the asset, metric, and time range
asset = 'btc'
metric = 'PriceUSD'
start_time = '2010-01-01'
end_time = datetime.today().strftime('%Y-%m-%d')  # Set end_time to today's date
frequency = '1d'

# Fetch the metric data for the specified asset and time range
logging.info("Fetching BTC PriceUSD...")
df = client.get_asset_metrics(
    assets=asset,
    metrics=[metric],
    frequency=frequency,
    start_time=start_time,
    end_time=end_time
).to_dataframe()

# Rename the 'PriceUSD' column to 'PriceUSD'
df = df.rename(columns={metric: 'PriceUSD'})

# Set 'time' as the index and normalize to remove the time component but keep it as a DatetimeIndex
df['time'] = pd.to_datetime(df['time']).dt.normalize()

# Remove timezone information, if any
df['time'] = df['time'].dt.tz_localize(None)

# Set 'time' as the index
df.set_index('time', inplace=True)

# Only keep the 'PriceUSD' data
btc_df = df[['PriceUSD']]

# Display
btc_df.info()
btc_df

2025-01-27 09:09:32 INFO     Fetching BTC PriceUSD...
2025-01-27 09:09:34 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2025-01-27 09:09:41 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2025-01-27 09:09:49 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5307 entries, 2010-07-18 to 2025-01-26
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PriceUSD  5307 non-null   Float64
dtypes: Float64(1)
memory usage: 88.1 KB


,PriceUSD
time,
2010-07-18,0.08584
2010-07-19,0.0808
2010-07-20,0.074736
2010-07-21,0.079193
2010-07-22,0.05847
...,...
2025-01-22,103791.128861
2025-01-23,104184.616744
2025-01-24,104716.124882


In [6]:
# Calculate Bitcoin daily returns
btc_df['BTC_Return'] = btc_df['PriceUSD'].pct_change()

# Select relevant columns and reset index for merging
btc_returns_df = btc_df[['BTC_Return']]
btc_returns_df = btc_returns_df.reset_index()
btc_returns_df.info()
btc_returns_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   time        5307 non-null   datetime64[ns]
 1   BTC_Return  5306 non-null   Float64       
dtypes: Float64(1), datetime64[ns](1)
memory usage: 88.2 KB


,time,BTC_Return
0,2010-07-18,<NA>
1,2010-07-19,-0.058714
2,2010-07-20,-0.075053
3,2010-07-21,0.059639
4,2010-07-22,-0.261679
...,...,...
5302,2025-01-22,-0.020727
5303,2025-01-23,0.003791
5304,2025-01-24,0.005102
5305,2025-01-25,0.001432


## Get S&P 500

In [8]:
# Fetch S&P 500 data
sp_data = yf.download('^GSPC', start='2010-07-18', progress=False)
sp_data['SP_Return'] = sp_data['Adj Close'].pct_change()

# Select relevant columns and reset index for merging
sp_df = sp_data[['SP_Return']].reset_index()

# Rename the Date column to match the Bitcoin data index
sp_df.rename(columns={'Date': 'time'}, inplace=True)

# Display the DataFrame information and content
sp_df.info()
sp_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3655 entries, 0 to 3654
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       3655 non-null   datetime64[ns]
 1   SP_Return  3654 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 57.2 KB


,time,SP_Return
0,2010-07-19,NaN
1,2010-07-20,0.011417
2,2010-07-21,-0.012820
3,2010-07-22,0.022513
4,2010-07-23,0.008220
...,...,...
3650,2025-01-21,0.008768
3651,2025-01-22,0.006138
3652,2025-01-23,0.005313
3653,2025-01-24,-0.002855


## Get Nasdaq-100

In [10]:
# Fetch Nasdaq-100 Index (NDX) data
ndx_data = yf.download('^NDX', start='2010-07-18', progress=False)
ndx_data['NDX_Return'] = ndx_data['Adj Close'].pct_change()

# Select relevant columns and reset index for merging
ndx_df = ndx_data[['NDX_Return']].reset_index()

# Rename the Date column to match the Bitcoin data index
ndx_df.rename(columns={'Date': 'time'}, inplace=True)

# Display information about the dataframe
ndx_df.info()

# Print the first few rows of the data
ndx_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3655 entries, 0 to 3654
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   time        3655 non-null   datetime64[ns]
 1   NDX_Return  3654 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 57.2 KB


,time,NDX_Return
0,2010-07-19,NaN
1,2010-07-20,0.011757
2,2010-07-21,-0.012696
3,2010-07-22,0.025202
4,2010-07-23,0.006591
...,...,...
3650,2025-01-21,0.005846
3651,2025-01-22,0.013284
3652,2025-01-23,0.002193
3653,2025-01-24,-0.005795


In [11]:
# Merge Bitcoin, S&P 500, and Nasdaq-100 returns on the 'time' column
combined_returns_df = (
    btc_df[['BTC_Return']]
    .join(sp_df.set_index('time'), how='inner')
    .join(ndx_df.set_index('time'), how='inner')
)

# Drop NaN values
returns_df = combined_returns_df.dropna()

returns_df = returns_df.astype('float64')

# Display information about the resulting DataFrame
returns_df.info()

# Show the first few rows of the data
returns_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3653 entries, 2010-07-20 to 2025-01-24
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BTC_Return  3653 non-null   float64
 1   SP_Return   3653 non-null   float64
 2   NDX_Return  3653 non-null   float64
dtypes: float64(3)
memory usage: 114.2 KB


,BTC_Return,SP_Return,NDX_Return
time,,,
2010-07-20,-0.075053,0.011417,0.011757
2010-07-21,0.059639,-0.012820,-0.012696
2010-07-22,-0.261679,0.022513,0.025202
2010-07-23,0.036311,0.008220,0.006591
2010-07-26,0.108020,0.011200,0.008009
...,...,...,...
2025-01-17,0.043198,0.009991,0.016590
2025-01-21,0.033131,0.008768,0.005846
2025-01-22,-0.020727,0.006138,0.013284


# Statistical Checks

In [13]:
def analyze_series(df, column):
    """
    Perform stationarity and autocorrelation tests on a given time series.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the time series.
    - column (str): The column name of the series to analyze.
    
    Returns:
    - dict: A dictionary containing ADF and Durbin-Watson test results.
    """
    results = {}

    # Perform Augmented Dickey-Fuller (ADF) test
    adf_result = adfuller(df[column])
    adf_statistic, p_value = adf_result[0], adf_result[1]
    results['ADF Statistic'] = adf_statistic
    results['p-value'] = p_value
    results['ADF Interpretation'] = (
        "Stationary" if p_value < 0.05 
        else "Non-stationary"
    )

    # Add intercept for regression
    df = df.copy()
    df['Intercept'] = 1

    # Fit a simple linear regression model to obtain residuals
    model = sm.OLS(df[column], df[['Intercept']]).fit()

    # Perform Durbin-Watson test for autocorrelation
    dw_statistic = durbin_watson(model.resid)
    results['Durbin-Watson Statistic'] = dw_statistic
    if 1.5 < dw_statistic < 2.5:
        results['DW Interpretation'] = "No significant autocorrelation"
    elif dw_statistic <= 1.5:
        results['DW Interpretation'] = "Positive autocorrelation detected"
    else:
        results['DW Interpretation'] = "Negative autocorrelation detected"

    return results

## Bitcoin Series 

In [15]:
# all historical data 
df = returns_df
all_btc_result = analyze_series(df, 'BTC_Return')
all_btc_result

{'ADF Statistic': -9.985210558850058,
 'p-value': 2.0641426748886218e-17,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 1.9657605773768414,
 'DW Interpretation': 'No significant autocorrelation'}

In [16]:
# last 8 years 
df = returns_df.loc['2017-01-01':]
recent_btc_result = analyze_series(df, 'BTC_Return')
recent_btc_result

{'ADF Statistic': -24.716714761558226,
 'p-value': 0.0,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.0464175061768115,
 'DW Interpretation': 'No significant autocorrelation'}

In [17]:
# post covid 
df = returns_df.loc['2022-01-01':]
post_covid_btc_result = analyze_series(df, 'BTC_Return')
post_covid_btc_result

{'ADF Statistic': -28.219475138760878,
 'p-value': 0.0,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.0391077090595737,
 'DW Interpretation': 'No significant autocorrelation'}

## S&P 500 Series

In [19]:
# all historical data 
df = returns_df
all_sp_result = analyze_series(df, 'SP_Return')
all_sp_result

{'ADF Statistic': -13.295554718769923,
 'p-value': 7.2180176152521115e-25,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.239416551036116,
 'DW Interpretation': 'No significant autocorrelation'}

In [20]:
# last 8 years 
df = returns_df.loc['2017-01-01':]
recent_sp_result = analyze_series(df, 'SP_Return')
recent_sp_result

{'ADF Statistic': -13.905994278671862,
 'p-value': 5.621714248820688e-26,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.3105141022856417,
 'DW Interpretation': 'No significant autocorrelation'}

In [21]:
# post covid 
df = returns_df.loc['2022-01-01':]
post_covid_sp_result = analyze_series(df, 'SP_Return')
post_covid_sp_result

{'ADF Statistic': -27.092568988684118,
 'p-value': 0.0,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 1.957876831859665,
 'DW Interpretation': 'No significant autocorrelation'}

## Nasdaq-100 Series

In [23]:
# all historical data 
df = returns_df
all_ndx_result = analyze_series(df, 'NDX_Return')
all_ndx_result

{'ADF Statistic': -13.599761469274013,
 'p-value': 1.9634338891585254e-25,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.2079967035091,
 'DW Interpretation': 'No significant autocorrelation'}

In [24]:
# last 8 years 
df = returns_df.loc['2017-01-01':]
recent_ndx_result = analyze_series(df, 'NDX_Return')
recent_ndx_result

{'ADF Statistic': -14.413666562299952,
 'p-value': 8.126017507501465e-27,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.2862369430999037,
 'DW Interpretation': 'No significant autocorrelation'}

In [25]:
# post covid 
df = returns_df.loc['2022-01-01':]
post_covid_ndx_result = analyze_series(df, 'NDX_Return')
post_covid_ndx_result

{'ADF Statistic': -17.304848457535744,
 'p-value': 5.60004897141409e-30,
 'ADF Interpretation': 'Stationary',
 'Durbin-Watson Statistic': 2.020329512819535,
 'DW Interpretation': 'No significant autocorrelation'}

# Beta 

In [27]:
def calculate_betas(df, dependent_col, independent_cols):
    """
    Calculate OLS regression betas for a dependent variable against multiple independent variables.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing the data.
    - dependent_col (str): Column name for the dependent variable.
    - independent_cols (list): List of column names for the independent variables.
    
    Returns:
    - dict: A dictionary with independent variable names as keys and their beta values as values.
    """
    betas = {}
    y = df[dependent_col]

    for col in independent_cols:
        # Add constant (intercept) to the independent variable
        X = sm.add_constant(df[col])
        
        # Fit OLS model
        model = sm.OLS(y, X).fit()
        
        # Extract and store the beta value for the independent variable
        betas[col] = round(model.params[col], 2)
    
    return betas

In [28]:
# all historical data 
df = returns_df
betas = calculate_betas(df, 'BTC_Return', ['SP_Return', 'NDX_Return'])
print("Entire historical data Beta values:", betas)

Entire historical data Beta values: {'SP_Return': 0.66, 'NDX_Return': 0.56}


In [29]:
# last 8 years 
df = returns_df.loc['2017-01-01':]
betas = calculate_betas(df, 'BTC_Return', ['SP_Return', 'NDX_Return'])
print("Last 8 years Beta values:", betas)

Last 8 years Beta values: {'SP_Return': 0.94, 'NDX_Return': 0.78}


In [30]:
# post covid 
df = returns_df.loc['2022-01-01':]
betas = calculate_betas(df, 'BTC_Return', ['SP_Return', 'NDX_Return'])
print("Post Covid Beta values:", betas)

Post Covid Beta values: {'SP_Return': 1.28, 'NDX_Return': 0.94}
